In [ ]:
import pandas
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tqdm import tqdm
import os
import cv2
from tensorflow.keras import layers, Sequential, datasets, Model
import warnings
warnings.filterwarnings('ignore')
# مازال يشكوا الصد حتي كرهت نفسه صاد الحروف و دالها

In [ ]:
dir_path = r"/kaggle/input/gan-getting-started/photo_jpg"
def load_images(folder_path, img_size = (128, 128)):
    X = []
    limit = 10000
    for img_name in tqdm(os.listdir(folder_path)[:limit]):
        img_path = os.path.join(folder_path, img_name)
        img_array = cv2.imread(img_path)
        img_array = cv2.resize(img_array, img_size)
        img_array = img_array[:,:,::-1]
        X.append(img_array / 255.0)
        if len(X) >= limit:
            break
    return np.array(X)

img_size = 64
X_train = load_images(dir_path, (img_size, img_size))
X_train.shape

In [ ]:
learning_rate = 0.0003
KL_coef = 0.0075
latent_dim = 64
batch_size = 100

In [ ]:
def show_images(images, title = None):
    plt.figure(figsize = (6, 6))
    for i in range(12):
        plt.subplot(3, 4, i + 1)
        plt.imshow(images[i])
        plt.title(title)
        plt.xticks([])
        plt.yticks([])
    plt.tight_layout()
    plt.show()
show_images(X_train)

In [ ]:
class VariationalAutoencoder(Model):
    def __init__(self, latent_dim = latent_dim):
        super().__init__()
        self.encoder = Sequential([
            layers.Conv2D(64, kernel_size = (3, 3), strides = 2, padding = 'same'),
            layers.BatchNormalization(),
            layers.LeakyReLU(0.2),
            
            layers.Conv2D(128, kernel_size = (3, 3), strides = 2, padding = 'same'),
            layers.BatchNormalization(),
            layers.LeakyReLU(0.2),
            
            layers.Conv2D(256, kernel_size = (3, 3), strides = 2, padding = 'same'),
            layers.BatchNormalization(),
            layers.LeakyReLU(0.2),
            
            layers.Flatten(),
            layers.Dense(2048),
            layers.BatchNormalization(),
            layers.LeakyReLU(0.2),
        ])
        
        self.get_mean = layers.Dense(latent_dim)
        self.get_logvar = layers.Dense(latent_dim)
        
        self.decoder = Sequential([
            layers.Dense(2048),
            layers.BatchNormalization(),
            layers.LeakyReLU(0.2),
            
            layers.Dense(256 * 8 * 8),
            layers.BatchNormalization(),
            layers.LeakyReLU(0.2),
            layers.Reshape((8, 8, 256)),
            
            layers.Conv2DTranspose(128, kernel_size = (3, 3), strides = 2, padding = 'same'),
            layers.BatchNormalization(),
            layers.LeakyReLU(0.2),
            
            layers.Conv2DTranspose(64, kernel_size = (3, 3), strides = 2, padding = 'same'),
            layers.BatchNormalization(),
            layers.LeakyReLU(0.2),
            
            layers.Conv2DTranspose(32, kernel_size = (3, 3), strides = 2, padding = 'same'),
            layers.BatchNormalization(),
            layers.LeakyReLU(0.2),

            layers.Conv2DTranspose(3, kernel_size = (3, 3), strides = 1, padding = 'same', activation = 'sigmoid'),
        ])
        
    def vae_loss(self, reconstructions, inputs, z_mean, z_log_var):
        reconstruction_loss = tf.reduce_mean(tf.square(reconstructions - inputs))
        kl_loss = -KL_coef * tf.reduce_mean(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
        total_loss = reconstruction_loss + kl_loss
        return kl_loss, reconstruction_loss, total_loss
        
    def encode(self, x):
        return self.encoder(x)
        
    def decode(self, x):
        return self.decoder(x)
        
    def sample(self, mean, logvar):
        eps = tf.random.normal(tf.shape(mean))
        return mean + tf.exp(logvar) * eps
    
    def call(self, x):
        encoded = self.encode(x)
        mean, logvar = self.get_mean(encoded), self.get_logvar(encoded)
        latent_space = self.sample(mean, logvar)
        decoded = self.decode(latent_space)
        return mean, logvar, decoded
    
    def train_step(self, batch):
        with tf.GradientTape() as tape:
            mean, logvar, y_pred = self.call(batch)
            kl_loss, reconstruction_loss, loss = self.vae_loss(y_pred, batch, mean, logvar)
        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        
        return {'kl_loss' : kl_loss, 'reconstruction_loss': reconstruction_loss, 'total_loss' : loss}

In [ ]:
model = VariationalAutoencoder()
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate))

In [ ]:
fixed_noise = tf.random.normal((60, latent_dim))

In [ ]:
model.fit(X_train, epochs = 1000)

In [ ]:
y_pred = model.decode(fixed_noise)
show_images(y_pred)